**Collection Mapper**

This module will assist MV collection system layout design and output a KML file.

> Troubleshooting: Run each cell starting from the beginning, and un-comment print statements to pinpoint the error. 

> Editing: Reference Networkx, SimpleKML, and Python documentation for help making changes. Ask ChatGPT for support as needed.

First-time users will need to install the following libraries: Networkx, Matplotlib, Numpy, tldraw, and SimpleKML. Remove all single quotes and run the following cell.

In [ ]:
'''
import pip
pip.main(["install", "networkx"])
pip.main(["install", "matplotlib"])
pip.main(["install", "numpy"])
pip.main(["install", "tldraw"])
pip.main(["install", "simplekml"])
'''

**Inputs**

**Data Preparation:**
- Load the WTG array KMZ into Global Mapper
- Add the substation as a point feature
- Create coordinate attributes (select all points using the Digitizer and right click > Attribute/Style Functions > Add Coordinates)
- Export CSV (open Layer Attributes to export and use only columns for Feature Name, X, and Y)
>Check sample CSV's if you have trouble with the data format

**In the cell below, enter the following information:**
1. CSV file containing location data: WTG & substation names in the first column, latitude in the second column, logitude in the third column
2. Substation name
3. Number of WTG's per feeder

In [102]:
datafile = 'wtgdata_nimbus.csv'

sub = 'S0'

capacity = 5

Clean the lat/long data exported from Global Mapper, and create a data structure containing the locations of each point.

In [ ]:
# Read CSV into a list
import csv
wtg_init = []
with open(datafile) as data:
    D = csv.reader(data, delimiter = ",")
    for row in D:
        wtg_init.append(row)
wtg_init = wtg_init[1:]
#print(wtg_init)

# Format lat/long data: check for quadrant and change type to float
# Create a dictionary data structure to hold coordinates for each point
# Ex. ['S00', '95.13234222? W', '42.00511659? N'] becomes {'S00': (-95.13234222, 42.00511659)}
if 'W' in wtg_init[1][1]:
    if 'N' in wtg_init[1][2]:
        for i in wtg_init:
            i[1] = -1*float(i[1].split('?')[0])
            i[2] = float(i[2].split('?')[0])
    else:
        for i in wtg_init:
            i[1] = -1*float(i[1].split('?')[0])
            i[2] = -1*float(i[2].split('?')[0])
else:
    if 'N' in wtg_init[1][2]:
        for i in wtg_init:
            i[1] = float(i[1].split('?')[0])
            i[2] = float(i[2].split('?')[0])
    else:
        for i in wtg_init:
            i[1] = float(i[1].split('?')[0])
            i[2] = -1*float(i[2].split('?')[0])

pos_dict = {}
for i in wtg_init:
    pos_dict[i[0]] = (float(i[1]),float(i[2]))
print(pos_dict)

if sub not in pos_dict:
    print('\033[1m' + 'Error: Substation name is not included in the input data.' + '\033[0m')

A weighted graph containing all possible connections in the WTG network is created. Optimal paths will be selected from this graph in the following steps.

In [ ]:
# Create a weighted graph of all possible turbine connections. 
# Weights are calculated as distances between points. 
wtg_network = []
for i in wtg_init:
    for j in wtg_init:
        flag = 0
        if i != j:
            distance =( (float(i[1])-float(j[1]))**2 + (float(i[2])-float(j[2]))**2 )**0.5
            for k in wtg_network:
                if k[0] == j[0] and k[1] == i[0]:
                    flag = 1
            if flag == 0:
                wtg_network.append([i[0],j[0],distance])

import networkx as nx
G = nx.Graph()
G.add_weighted_edges_from(wtg_network)

nx.draw_networkx(G, pos=pos_dict, with_labels=True)

**Sharma's Heuristic for a Capacitated Minimum Spanning Tree with no crossings**

_Sharma RL, El-Bardai MT (1970) Suboptimal communications network synthesis. Proceedings 1970 International Conference on Communications._

1. Calculate angle of all points relative to the substation, and sort
2. Group points in order
3. Add the substation to each group and construct minimum spanning trees
4. Find the version with the minimum weight

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math

# Function to calculate the angle from a center point
def calculate_angle(point, center):
    dx = point[0] - center[0]
    dy = point[1] - center[1]
    angle = math.atan2(dy, dx)
    return angle

# Function to find MST for a list of points
def find_mst_edges(point_list, graph):
    # Create a MST subgraph
    subgraph = graph.subgraph(point_list)
    mst = nx.minimum_spanning_tree(subgraph)
    
    # Extract edges from the MST
    mst_edges = []
    for u, v, data in mst.edges(data=True):
        mst_edges.append([u, v, data['weight']])
    return mst_edges

# Function to find the weight of a graph
def distance_calc(lines):
    total_weight = 0
    for i in lines:
        total_weight += i[2]
    total_weight *= 325000
    return total_weight

# Create a dictionary with angles from the center
center = pos_dict[sub]
angles_dict = {key: calculate_angle(value, center) for key, value in pos_dict.items()}

# Sort the dictionary by angle
sorted_angles = sorted(angles_dict.items(), key=lambda item: item[1])

# Create a list of keys from the sorted dictionary
sorted_keys = [item[0] for item in sorted_angles]
sorted_keys.remove(sub)

# Create list of list of lists containing iterations of Sharma's CMST = capacity
counter = capacity
sharma = []
while counter > 0:
    sharma_versions = [sorted_keys[i:i+capacity] for i in range(0, len(sorted_keys), capacity)]
    sorted_keys = sorted_keys[1:] + [sorted_keys[0]]
    #print(sorted_keys)
    #print(sharma_versions)
    sharma += [sharma_versions]
    counter -= 1
for i in sharma:
    for l in i:
        l.append(sub)

# Find the version with the minimum weight
s_weights = []
for j in sharma:
    s_lines = []
    for i in j:
        s_lines += find_mst_edges(i, G)
    s_weights += [distance_calc(s_lines)]
print(s_weights)

min_s = s_weights.index(min(s_weights))
sharma = sharma.pop(min_s)
print(sharma)
s_lines = []
for i in sharma:
    s_lines += find_mst_edges(i, G)

# Create and draw the graph
G_s = nx.Graph()
for i in sharma:
    G_s.add_weighted_edges_from(s_lines)
nx.draw_networkx(G_s, pos=pos_dict, with_labels=True, node_size=100, font_size = 8)

**Minimum Spanning Tree**

Connects all points along the shortest possible path. By default, the function uses Kruskal's Algorithm.

In [ ]:
print("Minimum Spanning Tree")
y = nx.minimum_spanning_tree(G)
nx.draw_networkx(y, pos=pos_dict, with_labels=True, node_size=100, font_size = 8)

**Optional Step**

Download PNG from above and upload to the whiteboard tool below to draw WTG groups. Adjust dimensions as needed.

In [ ]:
from tldraw import TldrawWidget
t = TldrawWidget(height = 500, width = 800)
t

**Input WTG Groups**

Calculate how many WTG's can be connected to each feeder.
For projects with multiple MPT's, put a balanced number of WTG's on each side.

>The names of the turbines and substation must match the names from the CSV file.

In [84]:
# Input collection system here:
collection = {}
collection['11A'] = [sub, '']
collection['11B'] = [sub, '']
collection['12A'] = [sub, '']
collection['12B'] = [sub, '']

collection['21A'] = [sub, '']
collection['21B'] = [sub, '']
collection['22A'] = [sub, '']
collection['22B'] = [sub, '']


# Check for incorrect WTG names and duplicates
col_inputs = []
for k in collection:
    for i in collection[k]:
        if i not in pos_dict:
            print(f'{i} is not in the input data.')
        if i not in col_inputs or i is sub:
            col_inputs.append(i)
        else:
            print(f'{i} has duplicate entries.')
#print(col_inputs)

The following cell calculates minimum spanning trees that connect each group of WTG's to the substation. An approximation of the total length is also provided. 

>Return to the previous cell and adjust the WTG groups until you are satisfied with the layout.

In [ ]:
def find_mst_edges(point_list, graph):
    # Create a subgraph with the given points
    subgraph = graph.subgraph(point_list)
    
    # Find the MST of the subgraph
    mst = nx.minimum_spanning_tree(subgraph)
    
    # Extract edges from the MST
    mst_edges = []
    for u, v, data in mst.edges(data=True):
        mst_edges.append([u, v, data['weight']])
    
    return mst_edges

mst_collections = {key: find_mst_edges(points, G) for key, points in collection.items()}
G_mst = nx.Graph()
for k in mst_collections:
    G_mst.add_weighted_edges_from(mst_collections[k])

nx.draw_networkx(G_mst, pos=pos_dict, with_labels=True, node_size=100, font_size = 8)

# Calculate total weight
total_weight = 0
for k in mst_collections:
    for i in mst_collections[k]:
        total_weight += i[2]
total_weight *= 325000
total_weight = round(total_weight, 2)

print('\033[1m' f"Approx. Total Cable Length = {total_weight} ft ± 10%" '\033[0m')
print(mst_collections)

The load_calculator() function is used to calculate the number of WTG's descending from each line segment. This is important for the naming convention of line segments so that the cable schedule will autimatically size the cables.

In [ ]:
def load_calculator(edges):
    Gx = nx.Graph()
    Gx.clear()
    
    # Add edges to the graph
    Gx.add_edges_from(edges)

    # Perform BFS starting from substation
    tree = list(nx.dfs_edges(Gx, source= sub))
    #print(tree)

    Gx.clear()
    Gx = nx.DiGraph()
    Gx.add_edges_from(tree)

    # Display the DFS traversal order
    dfs_order = [edge[1] for edge in tree]
    #print("DFS Traversal Order:", dfs_order)

    tree = [list(t) for t in tree]

    for i in dfs_order:
        for j in tree:
            if i == j[1]:
                descendants = len(nx.descendants(Gx,i)) + 1
                j.append(descendants)

    #print(tree)
    return tree

# Copy mst collection, and replace distance with # of WTG's
kml_drawer = mst_collections

for k in kml_drawer:
    kml_drawer[k] = [item[:2] for item in kml_drawer[k]]
 
for k in kml_drawer:
    #print(kml_drawer[k])
    kml_drawer[k] = load_calculator(kml_drawer[k])
print(kml_drawer)

**Create KML**

The kml_drawer and pos_dict data are used to draw line segments between points.

Folders are named after the feeder.

Line features are named "[Start point] [End point] [# of WTG's]" according to the cable schedule.

The output file will be located in the same folder. Running this script will overwrite the file, so rename it if you want to keep multiple drafts. 

In [89]:
import simplekml
kml = simplekml.Kml()

# Set colors for each feeder
colors = [
    simplekml.Color.tomato,
    simplekml.Color.thistle,
    simplekml.Color.cornflowerblue,
    simplekml.Color.magenta,
    simplekml.Color.gold,
    simplekml.Color.cyan, 
    simplekml.Color.orchid, 
    simplekml.Color.red,
    simplekml.Color.palegreen,
    simplekml.Color.yellow,
    simplekml.Color.olive, 
    simplekml.Color.blue
    ]
line_colors = {}
for k in kml_drawer:
    line_colors[k] = colors[0]
    colors = colors[1:] + [colors[0]]

for layer_desc, lines in kml_drawer.items():
    # Create a folder for each layer
    folder = kml.newfolder(name=layer_desc)
    
    # Iterate over each line in the layer
    for line in lines:
        start, end, attribute = line
        # Get the coordinates from pos_dict
        start_coords = pos_dict[start]
        end_coords = pos_dict[end]
        
        # Create a linestring for each line
        linestring = folder.newlinestring(
            name=f"{start} {end} {attribute}",
            coords=[(start_coords[0], start_coords[1]), (end_coords[0], end_coords[1])]
        )
        
        # Add a description or other attributes if needed
        linestring.description = f"{layer_desc}"
        linestring.style.linestyle.color = line_colors[layer_desc]
        linestring.style.linestyle.width = 2

# Save the KML to a file
kml.save("output.kml")
